In [1]:
import numpy as np
from numpy import allclose as eq, conj, array, complex128

V2 = np.sqrt(2)

# Basis

In [2]:
basis_0 = array([[1, 0]], dtype=complex).T
basis_1 = array([[0, 1]], dtype=complex).T

In [3]:
plus = (basis_0 + basis_1) / V2
minus = (basis_0 - basis_1) / V2

In [4]:
R = (basis_0 + 1j * basis_1) / V2
L = (basis_0 - 1j * basis_1) / V2

In [5]:
assert eq(conj(basis_0).T @ basis_1, 0)
assert eq(conj(plus).T @ minus, 0)
assert eq(conj(R).T @ L, 0)

# Hadamard Gate

In [6]:
H = array([[1, 1],
           [1, -1]], dtype=complex128) / V2

In [7]:
assert eq(conj(H) @ H, np.eye(2))
assert eq(H @ basis_0, plus)
assert eq(H @ plus, basis_0)

# Pauli Matrices

In [8]:
pauli_X = array([[0, 1],
                 [1, 0]], dtype=complex)

pauli_Y = array([[0, -1j],
                 [1j, 0]], dtype=complex)

pauli_Z = array([[1, 0],
                 [0, -1]], dtype=complex)

# Measurements

In [9]:
w, v = np.linalg.eig(pauli_Z)
for i in range(w.shape[0]):
    print (f"{i}: {str(np.round(w[i],10)):7s} -> {v[:,i]}")
    
assert eq(abs(v[:,[0]].conj().T @ basis_0), 1)
assert eq(abs(v[:,[1]].conj().T @ basis_1), 1)

0: (1+0j)  -> [1.+0.j 0.+0.j]
1: (-1+0j) -> [0.+0.j 1.+0.j]


In [10]:
w, v = np.linalg.eig(pauli_X)
for i in range(w.shape[0]):
    print (f"{i}: {str(np.round(w[i],10)):7s} -> {v[:,i]}")
    
assert eq(abs(v[:,[0]].conj().T @ plus), 1)
assert eq(abs(v[:,[1]].conj().T @ minus), 1)

0: (1+0j)  -> [0.70710678-0.j 0.70710678+0.j]
1: (-1+0j) -> [ 0.70710678+0.j -0.70710678-0.j]


In [11]:
w, v = np.linalg.eig(pauli_Y)
for i in range(w.shape[0]):
    print (f"{i}: {str(np.round(w[i],10)):7s} -> {v[:,i]}")

assert eq(abs(v[:,[0]].conj().T @ R), 1)
assert eq(abs(v[:,[1]].conj().T @ L), 1)

0: (1+0j)  -> [-0.        -0.70710678j  0.70710678+0.j        ]
1: (-1+0j) -> [0.70710678+0.j         0.        -0.70710678j]


## Expectation value

In [12]:
assert eq(plus.conj().T @ pauli_X @ plus, 1)

In [15]:
w, v = np.linalg.eig(pauli_Z)
v0, v1 = v[:,[0]], v[:,[1]]

proj_0 = v0 @ v0.T
proj_1 = v1 @ v1.T

In [16]:
q = H @ basis_0

p_0 = q.conj().T @ proj_0 @ q
p_1 = q.conj().T @ proj_1 @ q

print (p_0, p_1)

[[0.5+0.j]] [[0.5+0.j]]
